In this notebook I'm going to show an example illustrating my solution concept for Advent Of Code 2023 day 12.
For this purpose I will consider an input file of only one line:

`.??..?##? 1,3`

this example should result in a number of 4 different configurations of damaged springs in groups 1,3:

`.#...###.`
`.#....###`
`..#..###.`
`..#...###`

If you are unfamiliar with the concept of DFA / NFA (deterministic / nondeterministic finite automata), please look it up on YouTube now, as I'm certain there are many videos which are explaining it better than I ever could.

Starting off with our single input line now, we can see that we want to read one group of 1 spring `#` and one group of 3 springs `###`, separated by one or more dots, and possibly preceded and succeeded by an arbitrary number of dots.

If we draw this input limitation as an NFA, which can accept exactly these strings, it looks like this:

<br><img src="explanation.png" width="1200"><br>

This automaton has two types of states, dot-states and `#`-states.

As you can see this automaton takes an arbitrary amount of dots in each dot state (thats why the self referring loop), or it transitions into a dot state when succeeded by one and reading a dot, and a `#` char will always lead to advancing into a corresponding #-state. `?` however will enable any possible transition that the current state has anyways.

In code I created this automaton from the input numbers:

In [1]:
numbers = ["1","3"] # example

states = "."
for nr in numbers:
    for i in range(int(nr)):
        states += "#"
    states += "."
    
print([char for char in states])

['.', '#', '.', '#', '#', '#', '.']


We start off in the first (starting) dot-state, which is why this state has no transition conditions leading to it. Also we're in this state only once, so our current state dictionary looks like this:

In [2]:
states_dict = {0: 1}

If you are wondering why we have a dictionary instead of just one state: As this automaton is non-deterministic (in many cases there are two different states which we can transition into at the same time), we're using something similar to the power-set construction to execute an NFA deterministically. Please also look this concept up on YouTube if unfamiliar. Therefore we're holding all possible states that our machine can currently be in, which is part of the power set of all states. In our case we're extending this to a dictionary because we want to know how often we are in each state, as this corresponds to the number of possible "spring arrangements" in the end.

As we read each character of our input string `.??..?##?` we perform all possible transitions from each state that we're currently in.

In this example we're starting of with `.`, so our `states_dict` remains `{0: 1}`, because from the starting state there is only one transition with a dot, leading back to itself.

Our 2nd char however, the `?`, allows us to be either state 0 or 1, so our dict will look like this: `{0: 1, 1: 1}`. (State numbers are annotated in the bottom left corner of each box in the immage, they correspond to the string character index.)

On the 3rd char we read another `?`, so 
from state 0 we can transition to 0 and 1 again, 
from state 1 we have to transition to state 2,
so our state dict looks like this now: `{0: 1, 1: 1, 2: 1}`

On the 4th char it starts to get interesting: We read another dot, which means 
from state 0 we can only transition into 0, 
from state 1 into 2,
from state 2 into 2 as well.
We're adding all of these up, so now we're in state 0 once, and in state 2 twice: `{0: 1, 2: 2}`

After running the entire string our dictionary will look like this:
`{5: 2, 6: 2}`
Therefore we happened to end up on the last `#` state twice (`.#....###`,  `..#...###`) and we read one (or in theory also more) dot(s) afterwards (`.#...###.`, `..#..###.`,).
This adds up to 4 configurations, which is exactly what we were looking for.

Let's now get into the code:


In [3]:
text = ".??..?##?"
new_dict = {}
for char in text:
    for state in states_dict:
        if char == "?":
            if state + 1 < len(states):
                new_dict[state + 1] = new_dict.get(state + 1, 0) + states_dict[state]
            if states[state] == ".":
                new_dict[state] = new_dict.get(state, 0) + states_dict[state]

        elif char == ".":
            if state + 1 < len(states) and states[state + 1] == ".":
                new_dict[state + 1] = new_dict.get(state + 1, 0) + states_dict[state]
            if states[state] == ".":
                new_dict[state] = new_dict.get(state, 0) + states_dict[state]

        elif char == "#":
            if state + 1 < len(states) and states[state + 1] == "#":
                new_dict[state + 1] = new_dict.get(state + 1, 0) + states_dict[state]

    states_dict = new_dict
    new_dict = {}
    print(states_dict)

{0: 1}
{1: 1, 0: 1}
{2: 1, 1: 1, 0: 1}
{2: 2, 0: 1}
{2: 2, 0: 1}
{3: 2, 2: 2, 1: 1, 0: 1}
{4: 2, 3: 2, 1: 1}
{5: 2, 4: 2}
{6: 2, 5: 2}


As you can see, I implemented every transition between states depending on the character that we read in lines 5-19.
This snipped does exactly what we just did manually by simulating the automaton as `states` string and the currently held states as `states_dict`.
In the end all we need to do is add up our configurations for the last 2 (final/terminating states) and return the result:

In [4]:
print(states_dict.get(len(states) - 1, 0) + states_dict.get(len(states) - 2, 0))

4
